# Delta-RS Python Example

## STILL IN PROGRESS

This notebook shows how to write write and read Delta Lake tables without the need for Spark by leveraging the Delta-RS Python bindings. For more information about the delta-rs python bindings, seehttps://delta-io.github.io/delta-rs/python/

Click the Table of Contents button in the left JupyterLab sidebar (the button on the far left of this browser window that looks like a bulleted list) to see the types of examples provided. **Make sure to run all the cells above a given section, since most examples in this notebook depend on those above them**

In [1]:
from deltalake import DeltaTable
from deltalake.writer import write_deltalake
import pandas as pd
import pyarrow.fs as fs

import os

In [2]:
s3_options = {
    "REGION":"us-east-1",
    "AWS_S3_ALLOW_UNSAFE_RENAME":"true",
    "AWS_ALLOW_HTTP":"true", 
    # "ACCESS_KEY_ID": "jupyteraccesskey", 
    # "SECRET_ACCESS_KEY":"jupytersupersecretkey", 
    # "AWS_ENDPOINT_URL":"http://minio:9000",
}

## Credentials are provided in environment variables.
## Can override environment variables by uncommenting lines above

In [3]:
delta_table_uri = "s3://test/appl_stock_delta_table"

## Write a Delta Table

In [4]:
source_csv_path = "s3://test/appl_stock.csv"
df = pd.read_csv(source_csv_path)

In [5]:
## Add Month and Year for Partitioning
df["Month"] = df["Date"].str[5:7].astype(int)
df["Year"] = df["Date"].str[:4].astype(int)

In [6]:
write_deltalake(
    delta_table_uri, 
    df, 
    mode="overwrite",
    overwrite_schema=True,
    partition_by=["Year","Month"],
    storage_options=s3_options)

/usr/local/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


## Read a Delta Table

In [7]:
dt = DeltaTable(delta_table_uri, 
                storage_options=s3_options
               )

In [8]:
dt.version()

7

In [9]:
dt.files()[:3]

['Year=2010/Month=2/7-f88524b7-b77e-467d-b401-1a6b81129e9d-0.parquet',
 'Year=2010/Month=11/7-f88524b7-b77e-467d-b401-1a6b81129e9d-0.parquet',
 'Year=2010/Month=9/7-f88524b7-b77e-467d-b401-1a6b81129e9d-0.parquet']

In [10]:
dt.to_pandas().head()

,Date,Open,High,Low,Close,Volume,AdjClose,Month,Year
0,2010-02-01,192.369997,196.000000,191.299999,194.729998,187469100,25.229131,2,2010
1,2010-02-02,195.909998,196.319994,193.379993,195.859997,174585600,25.375533,2,2010
2,2010-02-03,195.169994,200.200003,194.420004,199.229994,153832000,25.812149,2,2010
3,2010-02-04,196.730003,198.370001,191.570005,192.050003,189413000,24.881912,2,2010
4,2010-02-05,192.630003,196.000000,190.850002,195.460001,212576700,25.323710,2,2010
